# Classification/Decision trees

In [36]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import pandas as pd
import joblib
import numpy as np
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore', category=FutureWarning)


Preprocessing the data

In [28]:
data = pd.read_csv('../Data Analysis/siren_data_train.csv')

# Preprocessing data
data.dropna(inplace=True)

df = pd.DataFrame()
distances = []
heard = []
for i in range(len(data)):
    point_a = np.array((float(data.iloc[i, 1]), float(data.iloc[i, 2])))
    point_b = np.array((float(data.iloc[i, 6]), float(data.iloc[i, 7])))
    distance = np.linalg.norm(point_a - point_b)
    distances.append(distance)
    heard.append(int(data.iloc[i, 4]))

df['distance'] = distances
df['heard'] = heard
data = data.assign(distance=df['distance'].values)

data.drop(["near_angle", "near_x", "near_y", "xcoor", "ycoor", "near_fid"], axis=1, inplace=True)

,heard,building,noise,in_vehicle,asleep,no_windows,age,distance
0,1,0,0,0,0,0,59,901.283517
1,1,0,0,0,0,0,29,972.006260
2,1,0,0,0,0,0,32,872.340924
3,1,0,0,0,0,0,36,257.804449
4,1,0,0,0,0,0,55,529.686791
...,...,...,...,...,...,...,...,...
5705,1,0,0,0,0,0,29,269.161517
5706,1,0,0,0,0,0,49,1076.700725
5707,1,0,0,0,0,0,62,549.581267
5708,1,0,0,0,0,0,37,353.136892


Applying the model to the data

In [29]:
# Define input and output
X = data.iloc[:, 2:]
y = data.iloc[:, 1].astype('category')

# Split the dataset into training and testing sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a Decision Tree Classifier
model = DecisionTreeClassifier()

# Train the classifier on the training data
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = model.predict(X_val)

# Evaluate the accuracy of the model
accuracy = accuracy_score(y_val, y_pred)
print(f"Accuracy: {accuracy}")

# Print the confusion matrix
pd.crosstab(y_val, y_pred, rownames=['Actual'], colnames=['Predicted'], margins=True)

# Save the model
#joblib.dump(model, 'model.joblib')




Accuracy: 0.6751313485113836


C:\Users\gabbe\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
C:\Users\gabbe\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\Users\gabbe\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
C:\Users\gabbe\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed